# Heterogeneous Autoregressive model

The HAR model assumes that the volatility (or other financial variables) is driven by components at different time scales (long-term, medium-term, and short-term).

## Load Data

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm

In [ ]:
file_path = "..\data\ALGO_daily.csv"
df = pd.read_csv(file_path)
df.rename(columns={'timestamp': 'Date'}, inplace=True)
df.set_index('Date', inplace=True)
df.head(2)

### Create Lagged lnRV

In [ ]:
df['lnRV_5D_lag'] = df['lnRV'].rolling(window=5).mean()
df['lnRV_7D_lag'] = df['lnRV'].rolling(window=7).mean()
df['lnRV_22D_lag'] = df['lnRV'].rolling(window=22).mean()
df['lnRV_30D_lag'] = df['lnRV'].rolling(window=30).mean()
df['lnRV_60D_lag'] = df['lnRV'].rolling(window=60).mean()
df.dropna(inplace=True)

## Train Test Split

In [ ]:
# Train-Test Split (80% train, 20% test)
train_size = int(0.8 * len(df))
test_size = len(df) - train_size
df_train, df_test = df.iloc[:train_size], df.iloc[train_size:]

print("Training Set:", df_train.shape, df_test.shape)
print("test period starts:", df_test.index[0])

In [ ]:
date_ticks = ['2024-01-10', '2024-04-01', '2024-07-01', '2024-10-01', '2025-01-01']

## 1-day ahead forecasting

### HAR(1,5,22)

$$
lnRV_t = \alpha + \beta_1 lnRV_{t-1} + \beta_2 lnRV_{5D, t-1} + \beta_3 lnRV_{22D, t-1} + \epsilon_t
$$
Where:
- $lnRV_t$ is the dependent variable (e.g., log realised volatility of return) at time \( t \).
- $\alpha$ is a constant term.
- $\beta_1, \beta_2, \beta_3$ are the coefficients for different time lags.
- $lnRV_{5D, t-1}$ and $lnRV_{22D, t-1}$ are the 1-day lag of the average log realised volatility of return over the past 5 days and 22 days respectively.
- $\epsilon_t$ is the error term.

In [ ]:
# Define dependent (y) and independent variables (X)
y_train = df_train['lnRV']
X_train = df_train[['lnRV_lag1', 'lnRV_5D_lag1', 'lnRV_22D_lag1']]
y_test = df_test['lnRV']
X_test = df_test[['lnRV_lag1', 'lnRV_5D_lag1', 'lnRV_22D_lag1']]

In [ ]:
# Add a constant to the independent value
X_train = sm.add_constant(X_train)

# Fit the OLS regression
model = sm.OLS(y_train, X_train).fit()

# Print summary of results
print(model.summary())

In [ ]:
# Predict y values
X_test = sm.add_constant(X_test)
y_pred = model.predict(X_test)

# Save the predicted values
y_pred.to_csv('../res/1D/HAR(1,5,22).csv', index=False)

In [ ]:
# Plot the predicted values against the actual values
plt.figure(figsize=(10, 5))
plt.plot(y_test, label='Actual')
plt.plot(y_pred, label='Predicted')
plt.title("HAR(1,5,22) Actual vs Predicted")
plt.xlabel("Date")
plt.xticks(ticks=date_ticks)
plt.legend()

# Save the plot
plt.savefig('../res/1D/HAR(1,5,22)_Actual_vs_Predicted.png')

### HAR(1,7,22)

$$
lnRV_t = \alpha + \beta_1 lnRV_{t-1} + \beta_2 lnRV_{7D, t-1} + \beta_3 lnRV_{30D, t-1} + \epsilon_t
$$
Where:
- $lnRV_t$ is the dependent variable (e.g., log realised volatility of return) at time \( t \).
- $\alpha$ is a constant term.
- $\beta_1, \beta_2, \beta_3$ are the coefficients for different time lags.
- $lnRV_{7D, t-1}$ and $lnRV_{30D, t-1}$ are the 1-day lag of the average log realised volatility of return over the past 7 days and 30 days respectively.
- $\epsilon_t$ is the error term.

In [ ]:
# Define dependent (y) and independent variables (X)
y_train = df_train['lnRV']
X_train = df_train[['lnRV_lag1', 'lnRV_7D_lag1', 'lnRV_30D_lag1']]
y_test = df_test['lnRV']
X_test = df_test[['lnRV_lag1', 'lnRV_7D_lag1', 'lnRV_30D_lag1']]

# Add a constant to the independent value
X_train = sm.add_constant(X_train)

# Fit the OLS regression
model = sm.OLS(y_train, X_train).fit()

# Print summary of results
print(model.summary())

In [ ]:
# Predict y values
X_test = sm.add_constant(X_test)
y_pred = model.predict(X_test)

# Save the predicted values
y_pred.to_csv('../res/1D/HAR(1,7,30).csv', index=False)

In [ ]:
# Plot the predicted values against the actual values
plt.figure(figsize=(10, 5))
plt.plot(y_test, label='Actual')
plt.plot(y_pred, label='Predicted')
plt.title("HAR(1,7,30) Actual vs Predicted")
plt.xlabel("Date")
plt.xticks(ticks=date_ticks)
plt.legend()

# Save the plot
plt.savefig('../res/1D/HAR(1,7,30)_Actual_vs_Predicted.png')

## 3-day ahead forecasting

### HAR(1,5,22)

In [ ]:
# Define dependent (y) and independent variables (X)
y_train = df_train['lnRV']
X_train = df_train[['lnRV_lag3', 'lnRV_5D_lag3', 'lnRV_22D_lag3']]
X_test = df_test[['lnRV_lag3', 'lnRV_5D_lag3', 'lnRV_22D_lag3']]

# Add a constant to the independent value
X_train = sm.add_constant(X_train)

# Fit the OLS regression
model = sm.OLS(y_train, X_train).fit()

# Print summary of results
print(model.summary())

In [ ]:
# Predict y values
X_test = sm.add_constant(X_test)
y_pred = model.predict(X_test)

# Save the predicted values
y_pred.to_csv('../res/3D/HAR(1,5,22).csv', index=False)

# Plot the predicted values against the actual values
plt.figure(figsize=(10, 5))
plt.plot(y_test, label='Actual')
plt.plot(y_pred, label='Predicted')
plt.title("HAR(1,5,22) Actual vs Predicted")
plt.xlabel("Date")
plt.xticks(ticks=date_ticks)
plt.legend()

# Save the plot
plt.savefig('../res/3D/HAR(1,5,22)_3D_Actual_vs_Predicted.png')

### HAR(1,7,22)

In [ ]:
# Define dependent (y) and independent variables (X)
y_train = df_train['lnRV']
X_train = df_train[['lnRV_lag3', 'lnRV_7D_lag3', 'lnRV_30D_lag3']]
X_test = df_test[['lnRV_lag3', 'lnRV_7D_lag3', 'lnRV_30D_lag3']]

# Add a constant to the independent value
X_train = sm.add_constant(X_train)

# Fit the OLS regression
model = sm.OLS(y_train, X_train).fit()

# Print summary of results
print(model.summary())

In [ ]:
# Predict y values
X_test = sm.add_constant(X_test)
y_pred = model.predict(X_test)

# Save the predicted values
y_pred.to_csv('../res/3D/HAR(1,7,30).csv', index=False)

# Plot the predicted values against the actual values
plt.figure(figsize=(10, 5))
plt.plot(y_test, label='Actual')
plt.plot(y_pred, label='Predicted')
plt.title("HAR(1,7,30) Actual vs Predicted")
plt.xlabel("Date")
plt.xticks(ticks=date_ticks)
plt.legend()

# Save the plot
plt.savefig('../res/3D/HAR(1,7,30)_3D_Actual_vs_Predicted.png')